# Imports

In [53]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from statistics import mean, stdev
import re
import string

# Question 1

## A
Upload the income_evaluation_cat.csv provided on canvas.  
The features in this data include workclass, education, race, and gender.  
The output variable is income and contains two categorical values (<=50k or >50k) indicating whether the income of an individual is less than/equal to \\$50,000 or greater than \\$50,000 respectively.  
Print the unique values of each variable in this data. 

In [12]:
data = pd.read_csv("income_evaluation_cat.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   workclass  32561 non-null  object
 1   education  32561 non-null  object
 2   race       32561 non-null  object
 3   gender     32561 non-null  object
 4   income     32561 non-null  object
dtypes: object(5)
memory usage: 1.2+ MB


In [13]:
data.head()

,workclass,education,race,gender,income
0,State-gov,Bachelors,White,Male,<=50K
1,Self-emp-not-inc,Bachelors,White,Male,<=50K
2,Private,HS-grad,White,Male,<=50K
3,Private,11th,Black,Male,<=50K
4,Private,Bachelors,Black,Female,<=50K


In [14]:
for col in data.columns:
    print(col)
    print(data[col].unique())

workclass
[' State-gov' ' Self-emp-not-inc' ' Private' ' Federal-gov' ' Local-gov'
 ' ?' ' Self-emp-inc' ' Without-pay' ' Never-worked']
education
[' Bachelors' ' HS-grad' ' 11th' ' Masters' ' 9th' ' Some-college'
 ' Assoc-acdm' ' Assoc-voc' ' 7th-8th' ' Doctorate' ' Prof-school'
 ' 5th-6th' ' 10th' ' 1st-4th' ' Preschool' ' 12th']
race
[' White' ' Black' ' Asian-Pac-Islander' ' Amer-Indian-Eskimo' ' Other']
gender
[' Male' ' Female']
income
[' <=50K' ' >50K']


## B
You will implement Naïve Bayes from scratch using Bayes’ rule.  
You can do your calculations in Python, but you would not use the sklearn package.  
Suppose that all the income_evaluation_cat.csv data you uploaded is the training data, classify a test instance, X = [“Private”, “Bachelors”, “White”, “Female”] into the class income<=50 or income>50k. Compute prior and posterior probabilities that are needed to classify the test instance. 
* 	What are the prior probabilities, P(income<=50) and P(income>50k)
* 
•	What are the unnormalized posterior probabilities, P(income\<=50/X) and P(income\>50k
* t

•	What are the normalized posterior probabilities, P(inc\ome<=50/X) and P(inc\ome>
* put

•	 Based on your normalized posterior probabilities, which class should the test instance be classifed into? 


In [27]:
data.iloc[2].gender

' Male'

In [16]:
P_income_minus_50k = data[data["income"] == " <=50K"].shape[0] / data.shape[0]
P_income_minus_50k

0.7591904425539756

In [17]:
P_income_over_50k = data[data["income"] == " >50K"].shape[0] / data.shape[0]
P_income_over_50k

0.2408095574460244

In [24]:
P_private_income_minus_50k = data[(data["income"] == " <=50K") & (data["workclass"] == " Private")].shape[0] / data[(data["income"] == " <=50K")].shape[0]
P_private_income_minus_50k

0.7173543689320389

In [29]:
P_private_income_over_50k = data[(data["income"] == " >50K") & (data["workclass"] == " Private")].shape[0] / data[(data["income"] == " >50K")].shape[0]
P_private_income_over_50k

0.6329549802321133

In [25]:
P_bachelors_income_minus_50k = data[(data["income"] == " <=50K") & (data["education"] == " HS-grad")].shape[0] / data[(data["income"] == " <=50K")].shape[0]
P_bachelors_income_minus_50k

0.35703883495145633

In [30]:
P_bachelors_income_over_50k = data[(data["income"] == " >50K") & (data["education"] == " HS-grad")].shape[0] / data[(data["income"] == " >50K")].shape[0]
P_bachelors_income_over_50k

0.21362071164392296

In [26]:
P_white_income_minus_50k = data[(data["income"] == " <=50K") & (data["race"] == " White")].shape[0] / data[(data["income"] == " <=50K")].shape[0]
P_white_income_minus_50k

0.8373381877022654

In [31]:
P_white_income_over_50k = data[(data["income"] == " >50K") & (data["race"] == " White")].shape[0] / data[(data["income"] == " >50K")].shape[0]
P_white_income_over_50k

0.9076648386685372

In [28]:
P_female_income_minus_50k = data[(data["income"] == " <=50K") & (data["gender"] == " Female")].shape[0] / data[(data["income"] == " <=50K")].shape[0]
P_female_income_minus_50k

0.38802588996763754

In [32]:
P_female_income_over_50k = data[(data["income"] == " >50K") & (data["gender"] == " Female")].shape[0] / data[(data["income"] == " >50K")].shape[0]
P_female_income_over_50k

0.15036347404667771

In [33]:
P_posterior_unnormalized_over = P_income_over_50k * P_private_income_over_50k * P_bachelors_income_over_50k * P_white_income_over_50k * P_female_income_over_50k
P_posterior_unnormalized_over

0.004443833326549051

In [34]:
P_posterior_unnormalized_minus = P_income_minus_50k * P_private_income_minus_50k * P_bachelors_income_minus_50k * P_white_income_minus_50k * P_female_income_minus_50k
P_posterior_unnormalized_minus

0.06317736934412253

In [35]:
P_posterior_normalized_over = P_posterior_unnormalized_over / (P_posterior_unnormalized_over + P_posterior_unnormalized_minus)
P_posterior_normalized_over

0.06571656745283554

In [36]:
P_posterior_normalized_minus = P_posterior_unnormalized_minus / (P_posterior_unnormalized_over + P_posterior_unnormalized_minus)
P_posterior_normalized_minus

0.9342834325471645

According to the normalized probability, the chosen class should be <= 50K

## C
Preprocess or transform the features in the income_evaluation_cat.csv data using an appropriate scaler in sklearn. You don’t need to transform the output variable; it should still work fine in a text format. 

In [37]:
data.head()

,workclass,education,race,gender,income
0,State-gov,Bachelors,White,Male,<=50K
1,Self-emp-not-inc,Bachelors,White,Male,<=50K
2,Private,HS-grad,White,Male,<=50K
3,Private,11th,Black,Male,<=50K
4,Private,Bachelors,Black,Female,<=50K


In [40]:
X = data.drop(["income"], axis=1).copy()
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X)

OneHotEncoder(handle_unknown='ignore')

In [43]:
X =enc.transform(X)
X

<32561x32 sparse matrix of type '<class 'numpy.float64'>'
	with 130244 stored elements in Compressed Sparse Row format>

## D
Randomly split the transformed input data and the output data into X_train, y_train, X_test and y_test using tools in sklearn. 

In [44]:
y = data["income"].copy()

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## E
Use an appropriate Naïve Bayes constructor in sklearn to construct and fit a Naïve Bayes model on the training data, then use the model to compute the accuracy score of the training and test set. 

In [48]:
nbb = BernoulliNB()
nbb.fit(X_train, y_train)

BernoulliNB()

In [50]:
y_pred = nbb.predict(X_test)
accuracy_score(y_test, y_pred)

0.7859555737537107

In [51]:
y_pred = nbb.predict(X_train)
accuracy_score(y_train, y_pred)

0.7851877851877852

Interestingly, the accuracy score is higher for the test dataset than for the training dataset. 

## Clean-up

In [52]:
del(data)
del(P_income_minus_50k)
del(P_income_over_50k)
del(P_private_income_minus_50k)
del(P_bachelors_income_minus_50k)
del(P_white_income_minus_50k)
del(P_private_income_over_50k)
del(P_bachelors_income_over_50k)
del(P_white_income_over_50k)
del(P_female_income_minus_50k)
del(P_female_income_over_50k)
del(P_posterior_unnormalized_over)
del(P_posterior_unnormalized_minus)
del(P_posterior_normalized_over)
del(P_posterior_normalized_minus)
del(enc)
del(X)
del(y)
del(X_train)
del(X_test)
del(y_train)
del(y_test)
del(nbb)

# Question 2

## A
Upload the income_evaluation_continuous.csv data provided on canvas.  
The features in this data include age, education_num, and hours_per_week. The output variable is income and contains two categorical values (<=50k or >50k) indicating whether the income of an individual is less than/equal to \\$50,000 or greater than \\$50,000 respectively.  
Compute the mean and standard deviation of each input variable such that the results are presented on the same table or data frame.  
You can call the .apply() function on the pandas DataFrame.  

In [53]:
data = pd.read_csv("income_evaluation_continuous.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   education_num   32561 non-null  int64 
 2   hours_per_week  32561 non-null  int64 
 3   income          32561 non-null  object
dtypes: int64(3), object(1)
memory usage: 1017.7+ KB


In [54]:
data.head()

,age,education_num,hours_per_week,income
0,39,13,40,<=50K
1,50,13,13,<=50K
2,38,9,40,<=50K
3,53,7,40,<=50K
4,28,13,40,<=50K


In [60]:
data.drop(["income"],axis=1).apply([mean, stdev])

,age,education_num,hours_per_week
mean,38.581647,10.080679,40.437456
stdev,13.640433,2.572720,12.347429


## B
Preprocess or transform the features in the income_evaluation_cont.csv data using an appropriate scaler in sklearn.  
You don’t need to transform the output variable; it should still work fine in a text format. 

In [62]:
X = data.drop(["income"], axis=1).copy()
esc = StandardScaler()
esc.fit(X)
X = esc.transform(X)

In [63]:
X

array([[ 0.03067056,  1.13473876, -0.03542945],
       [ 0.83710898,  1.13473876, -2.22215312],
       [-0.04264203, -0.42005962, -0.03542945],
       ...,
       [ 1.42360965, -0.42005962, -0.03542945],
       [-1.21564337, -0.42005962, -1.65522476],
       [ 0.98373415, -0.42005962, -0.03542945]])

## C
Randomly split the input and output data into X_train, y_train, X_test and y_test using tools in sklearn. 

In [64]:
y = data["income"].copy()

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## D
Use an appropriate Naïve Bayes constructor in sklearn to construct and fit a Naïve Bayes model on the training data, then use the model to compute the accuracy score of the training and test set. 

In [72]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

In [73]:
y_pred = gnb.predict(X_test)

In [74]:
accuracy_score(y_test, y_pred)

0.7988535162247927

## Clean up

In [75]:
del(X)
del(y)
del(data)
del(esc)
del(X_train)
del(X_test)
del(y_train)
del(y_test)
del(y_pred)
del(gnb)

# Question 3
You will now implement a Naïve Bayes for text classification to detect fake or true news. 

## A
Upload the True.csv data provided on canvas into Python.  
You will create a new data frame by selecting the “title” and “text” columns, then, adding a new column called “news_type” where all the values on this new column are “True”.  
So, your new data frame should have three columns; “title”, “text” and “news_type”. 

In [122]:
data_True = pd.read_csv("True.csv")
data_True = data_True[["title", "text"]]
data_True["news_type"] = np.full((data_True.shape[0],1),"True")
data_True

,title,text,news_type
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,True
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,True
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,True
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,True
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,True
...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,True
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",True
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,True
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,True


## B
Upload the Fake.csv data provided on canvas into Python.  
You will create a new data frame by selecting the “title” and “text” columns, then, adding a new column called “news_type” where all the values on this new column are “Fake”.  
So, your new data frame should have three columns; “title”, “text” and “news_type”.

In [123]:
data_Fake = pd.read_csv("Fake.csv")
data_Fake = data_Fake[["title", "text"]]
data_Fake["news_type"] = np.full((data_Fake.shape[0],1),"Fake")
data_Fake

,title,text,news_type
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,Fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,Fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",Fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",Fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,Fake
...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Fake
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Fake
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Fake
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Fake


## C
Merge the data frame in a) and b) so that one of the data frames is stacked vertically on top of the other.  
Combine the text in the “title” and “text” columns of the merged data frame into another column called “news”.  
Drop the “title” and “text” columns so that your final data frame is has only two columns, “news” and “news_type”.  
Print the first five rows of your final data frame. 

In [124]:
data = pd.concat([data_True, data_Fake])
data["news"] = data["title"].astype(str) + data["text"]
data = data.drop(["title","text"], axis=1)
data.head(n=5)

,news_type,news
0,True,"As U.S. budget fight looms, Republicans flip t..."
1,True,U.S. military to accept transgender recruits o...
2,True,Senior U.S. Republican senator: 'Let Mr. Muell...
3,True,FBI Russia probe helped by Australian diplomat...
4,True,Trump wants Postal Service to charge 'much mor...


## D
Preprocess your data by cleaning the textual data in the “news” column and removing the stop words, special characters, punctuations, etc especially at the beginning and end of each word.  
You can display any messy news text before you clean the data, then display the messy news text again after cleaning the data to see if your data cleaning worked well.  
Also, drop instances where the news text is less than 50 words. 

In [125]:
stopwords = set(nltk.corpus.stopwords.words('english'))

In [126]:
# All news that contain stopwords
data[data["news"].str.contains("|".join(stopwords))]

,news_type,news
0,True,"As U.S. budget fight looms, Republicans flip t..."
1,True,U.S. military to accept transgender recruits o...
2,True,Senior U.S. Republican senator: 'Let Mr. Muell...
3,True,FBI Russia probe helped by Australian diplomat...
4,True,Trump wants Postal Service to charge 'much mor...
...,...,...
23476,Fake,McPain: John McCain Furious That Iran Treated ...
23477,Fake,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...
23478,Fake,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...
23479,Fake,How to Blow $700 Million: Al Jazeera America F...


In [127]:
data["news"] = data["news"].apply(lambda x: ' '.join([item for item in x.split() if item not in stopwords]))
data

,news_type,news
0,True,"As U.S. budget fight looms, Republicans flip f..."
1,True,U.S. military accept transgender recruits Mond...
2,True,Senior U.S. Republican senator: 'Let Mr. Muell...
3,True,FBI Russia probe helped Australian diplomat ti...
4,True,Trump wants Postal Service charge 'much more' ...
...,...,...
23476,Fake,McPain: John McCain Furious That Iran Treated ...
23477,Fake,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...
23478,Fake,Sunnistan: US Allied ‘Safe Zone’ Plan Take Ter...
23479,Fake,How Blow $700 Million: Al Jazeera America Fina...


In [128]:
# Removing special characters and punctuation
regex = "[^A-Za-z0-9 ]+"

In [129]:
data["news"] = data["news"].apply(lambda x: re.sub(regex, '', x))
data

,news_type,news
0,True,As US budget fight looms Republicans flip fisc...
1,True,US military accept transgender recruits Monday...
2,True,Senior US Republican senator Let Mr Mueller jo...
3,True,FBI Russia probe helped Australian diplomat ti...
4,True,Trump wants Postal Service charge much more Am...
...,...,...
23476,Fake,McPain John McCain Furious That Iran Treated U...
23477,Fake,JUSTICE Yahoo Settles Email Privacy Classactio...
23478,Fake,Sunnistan US Allied Safe Zone Plan Take Territ...
23479,Fake,How Blow 700 Million Al Jazeera America Finall...


In [130]:
# Removing news with fewer than 50 characters
for i in range(data.shape[0]):
    if len(data.iloc[i]["news"]) < 50:
        print(data.iloc[i]["news"])

Graphic Supreme Court roundup
WATTERS WORLD Do ObamacareHow work Video
ITS NOT A MUSLIM BAN You Big Dummies VIDEO
LIVE FEED INAUGURATION 2017
BEST TWEET OF THE WEEK Hillary For Sale
WATCH Life Accordion To Trump VIDEO
YES OBAMAThere Is A Magic Wand Video
WHERES HILLARY CLINTON SPOTTED Dining Alone
TWEET OF THE DAY HillaryThe Price Of Failure
OBAMA UNHINGED AND ANGRY He doin nothin Video
AMAZING ENTRANCE Trump Does Les Deplorables Video
The MAKE AMERICA GREAT AGAIN DoItYourself Kit
MUST WATCH VIDEO BUSH EERILY WARNS US ABOUT ISIS
VIDEO LIBERAL PUNDIT OBAMA WORST POTUS EVER
WATTERS WORLD Do ObamacareHow work Video
Judge Napolitano Slams SCOTUS Ruling Obamacare
WATTERS WORLD Do ObamacareHow work Video


In [131]:
# Removing news with less that 50 characters
data = data[data["news"].str.len() >= 50]
data

,news_type,news
0,True,As US budget fight looms Republicans flip fisc...
1,True,US military accept transgender recruits Monday...
2,True,Senior US Republican senator Let Mr Mueller jo...
3,True,FBI Russia probe helped Australian diplomat ti...
4,True,Trump wants Postal Service charge much more Am...
...,...,...
23476,Fake,McPain John McCain Furious That Iran Treated U...
23477,Fake,JUSTICE Yahoo Settles Email Privacy Classactio...
23478,Fake,Sunnistan US Allied Safe Zone Plan Take Territ...
23479,Fake,How Blow 700 Million Al Jazeera America Finall...


In [132]:
# Removing duplicate news
data = data.drop_duplicates(subset="news")
data

,news_type,news
0,True,As US budget fight looms Republicans flip fisc...
1,True,US military accept transgender recruits Monday...
2,True,Senior US Republican senator Let Mr Mueller jo...
3,True,FBI Russia probe helped Australian diplomat ti...
4,True,Trump wants Postal Service charge much more Am...
...,...,...
22698,Fake,The White House The Theatrics Gun Control21st ...
22699,Fake,Activists Terrorists How Media Controls Dictat...
22700,Fake,BOILER ROOM No Surrender No Retreat Heads Wil...
22701,Fake,Federal Showdown Looms Oregon After BLM Abuse ...


## E
Transform the input text data into feature vectors where the entries of the feature vectors are term-frequency-inverse-document-frequency.  
Use the TfidfVectorizer() in sklearn. 

In [133]:
vectorizer = TfidfVectorizer()
a = vectorizer.fit_transform(data["news"])
a

<39071x251582 sparse matrix of type '<class 'numpy.float64'>'
	with 6887829 stored elements in Compressed Sparse Row format>

In [134]:
temp = data.copy()
for i in range(data.shape[0]):
    temp.iloc[i]["news"] = a[i]
temp

,news_type,news
0,True,"(0, 249770)\t0.02465021851116228\n (0, 1893..."
1,True,"(0, 110781)\t0.0343572114326606\n (0, 92431..."
2,True,"(0, 111071)\t0.056331516709626425\n (0, 109..."
3,True,"(0, 145801)\t0.04549063378009072\n (0, 1157..."
4,True,"(0, 9129)\t0.019663964021919812\n (0, 22349..."
...,...,...
22698,Fake,"(0, 97360)\t0.058753510792725436\n (0, 1793..."
22699,Fake,"(0, 48299)\t0.022413395930641213\n (0, 9409..."
22700,Fake,"(0, 208989)\t0.13729258076743073\n (0, 1031..."
22701,Fake,"(0, 228036)\t0.01618954860512946\n (0, 3709..."


In [139]:
# For reasons unknown, assigning it directly creates memory issues requesting over 60 Gb, 
# but assigning it to a temporary dataframe, and then copying that dataframe to the original dataframe solves the issue
data = temp.copy()
data

,news_type,news
0,True,"(0, 249770)\t0.02465021851116228\n (0, 1893..."
1,True,"(0, 110781)\t0.0343572114326606\n (0, 92431..."
2,True,"(0, 111071)\t0.056331516709626425\n (0, 109..."
3,True,"(0, 145801)\t0.04549063378009072\n (0, 1157..."
4,True,"(0, 9129)\t0.019663964021919812\n (0, 22349..."
...,...,...
22698,Fake,"(0, 97360)\t0.058753510792725436\n (0, 1793..."
22699,Fake,"(0, 48299)\t0.022413395930641213\n (0, 9409..."
22700,Fake,"(0, 208989)\t0.13729258076743073\n (0, 1031..."
22701,Fake,"(0, 228036)\t0.01618954860512946\n (0, 3709..."


In [142]:
data.iloc[0]["news"]

<1x251582 sparse matrix of type '<class 'numpy.float64'>'
	with 326 stored elements in Compressed Sparse Row format>

In [140]:
del(temp)

## F
Spit the feature vectors and the output variable into into X_train, y_train, X_test and y_test, you can let the test set be 30% of the entire data. 

## G
Fit an appropriate Naïve Bayes model and compute the training and test accuracy of the model.  
Is there overfitting?

## H
Fit a Naïve Bayes using cross validation and print the average cross validation score as well as the standard deviation of the cross-validation scores. 

## I
Select some hypermeters of your choice and tune using the grid search cross validation.  
Use some other hyperparameters than those used in class examples. 